To start, upload:
- `.tar.bz2` file containing data from most recent scrape
- `data.py`
- `model.py`
- `train.py`
- `requirements.txt`

In [4]:
!mkdir data/
!tar -xjvf /content/11-25-20_21-1500.tar.bz2 -C data/

meme_data.tsv


In [ ]:
!pip3 install -r requirements.txt

The following code block is all that you should need to run to test `train.py`, `model.py`, and `data.py`.

In [61]:
!python train.py --gpus 1
# !python train.py --gpus 1 --auto_scale_batch_size

2020-12-11 08:13:31.892710: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'multiple_choice_head.summary.bias', 'lm_head.weight', 'multiple_choice_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                 | Params
-----------------------------------------------
0 | model | GPT2DoubleHeadsModel |

The following code blocks test `data.py` and `model.py` in greater detail.

In [2]:
import pickle

In [1]:
import data

In [3]:
dataset = data.MemesDataModule()
dataset.prepare_data() # this generates a pickle
dataset.setup() # generates dataloaders

In [4]:
with open("/content/data/data.pickle", 'rb') as handle:
  test_pickle_data = pickle.load(handle)

In [5]:
print(type(test_pickle_data))
for i in range(10):
  print(test_pickle_data[i])
  # print(test_pickle_data[i]['caption']['input_ids'].size())

x = test_pickle_data[0:9]

<class 'list'>
always has been<|SEP|>Dysfunctional families:
 always has been
Wait thanksgiving is cancelled?<|endoftext|>
always has been<|SEP|>look at this bullet
Look at the view<|endoftext|>
always has been<|SEP|>never had any
wait theres no context?<|endoftext|>
always has been<|SEP|>always has been
Wait the wat the wat its a Thicc boi<|endoftext|>
always has been<|SEP|>Always has been
Wait, it's all sword?<|endoftext|>
always has been<|SEP|>always have
wait, free trials ask for your criedit card?<|endoftext|>
always has been<|SEP|>never has been
wait no free robux?<|endoftext|>
always has been<|SEP|>Always has been
so just being polite to a girl isn't simping<|endoftext|>
always has been<|SEP|>Always has been. Some upvote beggar lying about asking their crush out
Wait, this guy is lying for upvotes?<|endoftext|>
always has been<|SEP|>always has been
wait its memes?<|endoftext|>


In [6]:
print(dataset.tokenizer)

PreTrainedTokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'sep_token': '<|SEP|>', 'pad_token': '<|endoftext|>'})


In [7]:
print(type(x[0]))

<class 'str'>


In [10]:
a = dataset.tokenizer(x, return_tensors='pt', padding=True, truncation=True)
print(a['input_ids'].size())

torch.Size([9, 31])


In [8]:
train_dl = dataset.train_dataloader()
val_dl = dataset.val_dataloader()
test_dl = dataset.test_dataloader()

In [9]:
print(train_dl)

In [7]:
from model import GPT2

In [ ]:
GPT2_model = GPT2(lr=0.02, num_warmup_steps=1, num_training_steps=1, weight_decay=0.0)

Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight', 'multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
GPT2_model.configure_optimizers()

([AdamW (
  Parameter Group 0
      amsgrad: False
      betas: (0.9, 0.999)
      eps: 1e-08
      initial_lr: 0.02
      lr: 0.0
      weight_decay: 0.0
  )], [<torch.optim.lr_scheduler.LambdaLR at 0x7fe2bf8995f8>])

In [ ]:
one_item = next(iter(train_dl))['caption']
GPT2_model.forward(one_item)

GPT2DoubleHeadsModelOutput([('logits',
                             tensor([[[ -31.7896,  -30.7168,  -32.1722,  ...,  -39.7643,  -31.6573,
                                          1.1465],
                                      [ -63.3437,  -60.9143,  -61.4364,  ...,  -72.0760,  -62.4938,
                                          2.9376],
                                      [ -55.8653,  -56.3047,  -56.5112,  ...,  -64.2611,  -58.4208,
                                          2.4771],
                                      ...,
                                      [-103.1343, -103.0198, -103.0234,  ..., -109.8136, -103.0093,
                                          3.2291],
                                      [ -79.7854,  -82.7699,  -83.3494,  ...,  -93.6098,  -81.2983,
                                          2.9676],
                                      [ -82.7513,  -76.0932,  -78.2796,  ...,  -96.2631,  -84.0178,
                                          5.1474]]], grad_fn=<U

In [ ]:
GPT2_model.train_step(one_item, _)
GPT2_model.val_step(one_item, _)
GPT2_model.test_step(one_item, _)

The following code blocks test the capability of the model on distributed systems (they should all perform successfully).

In [ ]:
import pickle
with open('test.pickle', 'wb') as handle:
  pickle.dump(GPT2_model, handle)

In [ ]:
with open('test2.pickle', 'wb') as handle:
  pickle.dump(train_dl, handle)
with open('test3.pickle', 'wb') as handle:
  pickle.dump(val_dl, handle)
with open('test4.pickle', 'wb') as handle:
  pickle.dump(test_dl, handle)